<a href="https://colab.research.google.com/github/SakuraPie/ai-model/blob/main/sovits_f0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[sovits合集导航](https://github.com/IceKyrin/sovits_guide)**

[sovits_f0 git](https://github.com/innnky/so-vits-svc)

1、**本colab为Rcell版sovits2.0(f0分支)，仅支持多人模型，模型仅在三个一键脚本内部互通**

2、可以使用**预模型（已替换为22050hz）**节省训练时间，id范围0-7，**最多八个人物**，按提示操作即可；**预模型**可以只放一个人物，从头训练至少2个。

3、模型自动存储在谷歌盘内，请**定时清理空间**，以免盘满丢失进度。

4、**报错**请**先检查数据集和路径**，格式参考数据集制作colab，本colab**经过多人测试通过，已高度自动化**；按提示修改文件名、路径即可，**除非知道自己在做什么**，**不要自己发明新代码、手动创建新路径**

5、请确保网络连通性良好，报错也可能是因为网络原因

6、只需要保证**最新生成的**D、G.pth,config.json在，即可继续训练；每次**断点继续**训练，都是**步骤完全一样**的从头开始；**请记录自己的初始操作**，每次保持一致

7、对于**有一点点基础**的，**断点继续训练**：首次成功运行之后，保存/content/sovits_f0_train/路径下，**configs里的xxx.json**、**filelist里的train/val.txt**、云盘里**最新的G、D.pth**、**dataset里的各人物数据文件夹**，手动上传一下，保证这几个和首次运行一致即可。**不明白就参考第六步，每次都和第一次完全相同的操作即可**

前置：[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n#scrollTo=xx2oAf90btEy)

后置：[一键合成](https://colab.research.google.com/drive/1F3VpHCi9eridGw1F1hbqR7qhXGKuSCus#scrollTo=vjkgBV7j2cVJ)




In [ ]:
# 查看显卡
!nvidia-smi

In [ ]:
#@title 准备
#@markdown 定义工具函数 `run_command` `run_command_by_line` `get_symbols` 和 `get_tensorboard_showing`
# forked from https://www.endpointdev.com/blog/2015/01/getting-realtime-output-using-python/
import os
import subprocess
def run_command(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    try:
      process = subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      out, err = process.communicate()
    except:
      pass
    print_pipe(out)
    print_pipe(err)
    rc = process.poll()
    return rc

def run_command_by_line(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    with subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
      while process.poll() is None:
        print_pipe(process.stdout.readline())
      [print_pipe(line) for line in process.stderr.readlines()]
    return

def get_tensorboard_showing(logdir):
    from multiprocessing import Process
    from tensorboard import notebook
    import tensorflow as tf
    import time

    def run_tb():
        run_command_by_line(["tensorboard","--reload_interval", "30",  "--logdir", logdir, "--bind_all"])
    
    def monitor_tb():
        while True:
            try:
                notebook.display(height=998)
                break
            except Exception as e:
                print(e)
                time.sleep(3)

    if param_enable_tb:
        Process(target=run_tb).start()
        Process(target=monitor_tb).start()

In [ ]:
#@title 下载依赖库
colab_save_space = True #@param {type:"boolean"}
os.chdir('/content')
run_command_by_line(["git", "clone", "https://github.com/IceKyrin/sovits_f0_train.git", "-b", "main" if colab_save_space else "main"])
os.chdir('/content/sovits_f0_train')
!pip install -r requirements.txt
!sudo apt-get install espeak -y
!sudo apt-get install p7zip-full p7zip-rar
!pip install demjson

In [ ]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 解压数据集
!mkdir /content/sovits_f0_train/dataset

#@markdown 每次解压**一个**，确认完成后**修改名称**、**依次**解压**所有**压缩包

#@markdown 数据集制作时，确定的人物名称(前缀加"out_",不带zip)
DATASETNAME = "out_qiu"  #@param {type:"string"}
#@markdown 压缩包路径(这个名字是接着上一篇的输出zip)
ZIP_PATH = "/content/drive/MyDrive/dataset/out_qiu.zip"  #@param {type:"string"}
DATASETPATH = "/content/sovits_f0_train/" + DATASETNAME
%cd /content/sovits_f0_train/dataset
!cp {ZIP_PATH} {DATASETNAME}.zip
!unzip -q {DATASETNAME}.zip

In [ ]:
#@title 生成filelist
import re
import demjson
!mkdir /content/sovits_f0_train/filelist
%cd /content/sovits_f0_train/


def replace_id(f_list, sp_id):
    if re.search('wav[|](\d+)[|]', f_list[0]):
        split_char = "wav|%s|" % re.search('wav[|](\d+)[|]', f_list[0]).group(1)
    else:
        split_char = "wav|"
    f_list = [x.replace(split_char, f"wav|{sp_id}|") for x in f_list]
    return f_list


# data文件夹下放，各个 工程名/wavs/train(val)(这层可以不要)/xxx.wav
# 自动读取每个工程文件夹下的train(val).txt(xxx.wav|无符号文本)
#@markdown 默认为0，总体偏移量，不明白别改
pre = 0  #@param {type:"string"} 
dataset_path = "./dataset"  # 数据合集目录
file_train_name = "train.txt"
file_val_name = "val.txt"
#@markdown 人物名(**必须与数据集制作时人物名相同，个数也相同，英文逗号隔开**)，**预模型**可以只放一个

#@markdown 就是已经在dataset文件夹下解压出的数据文件夹的名称，也是人物名称

#@markdown id即为此处人物排序，从0自动开始赋值
speakers = "qiu,x,y,z" #@param {type:"string"}
speakers = [speaker.strip() for speaker in speakers.split(",")]

print(speakers)

train_f = open("./filelist/train.txt", "w", encoding="utf-8")
val_f = open("./filelist/val.txt", "w", encoding="utf-8")
pre = int(pre)
for i in range(pre, len(speakers) + pre):
    with open(f"{dataset_path}/{speakers[i]}/{file_train_name}", "r", encoding="utf-8") as f:
        file_list = f.readlines()
        train_f.writelines(replace_id(file_list, i))
    with open(f"{dataset_path}/{speakers[i]}/{file_val_name}", "r", encoding="utf-8") as f:
        file_list = f.readlines()
        val_f.writelines(replace_id(file_list, i))
train_f.close()
val_f.close()


In [ ]:
#@title 生成配置文件 在configs下，记得自己下载保留一份，下次用
#@markdown 本colab内可选下载的**预模型已替换为22050hz**，id为0-7可用

#@markdown **预模型请直接使用下一步下载的nyarumul.json，id0-7可用，json无需修改**

#@markdown **预模型的json可以不改动，只放一个id的数据进去**

# forked from https://github.com/CjangCjengh/vits/blob/main/configs/japanese_ss_base2.json

#@markdown 44100采样率，训练时长要求高，**和数据集制作时统一采样率**
high_sample_rate = False #@param {type:"boolean"}
#@markdown 配置文件名称
json_filename = "model.json" #@param {type:"string"}
#@markdown 训练次数
hparams_epochs = 3000 #@param {type:"integer"}
#@markdown 每隔多少次step保存一次断点
hparams_eval_interval = 2000 #@param {type:"integer"}
#@markdown 单次step的文件数（建议在16以内，若使用44100采样率，在8以内）
hparams_batch_size = 16 #@param {type:"integer"}
#@markdown 训练集文件列表
hparams_training_files = "/content/sovits_f0_train/filelist/train.txt" #@param {type:"string"}
#@markdown 验证集文件列表
hparams_validation_files = "/content/sovits_f0_train/filelist/val.txt"#@param {type:"string"}
#@markdown 人物名，多个人物用英文逗号隔开，必须**与上面填写的相同**
hparams_speaker = "qiu,x,y,z" #@param {type:"string"}
#@markdown 模型名
hparams_model_name = "model" #@param {type:"string"}

speakers = [speaker.strip() for speaker in hparams_speaker.split(",")]
print("speakers: ")
for i, speaker in enumerate(speakers):
  print("\t{a}: {b}".format(a=i, b=speaker))
training_json = {
  "train": {
    "log_interval": 200,
    "eval_interval": hparams_eval_interval,
    "seed": 1234 ,
    "epochs": hparams_epochs,
    "learning_rate": 2e-4,
    "betas": [0.8, 0.99],
    "eps": 1e-9,
    "batch_size": hparams_batch_size,
    "fp16_run": True,
    "lr_decay": 0.999875,
    "segment_size": 16384 if high_sample_rate else 8192,
    "init_lr_ratio": 1,
    "warmup_epochs": 0,
    "c_mel": 45,
    "c_kl": 1.0
  },
  "data": {
    "training_files": hparams_training_files,
    "validation_files": hparams_validation_files,
    "text_cleaners":["english_cleaners2"],
    "max_wav_value": 32768.0,
    "sampling_rate": 44100 if high_sample_rate else 22050,
    "filter_length": 2048 if high_sample_rate else 1024,
    "hop_length": 512 if high_sample_rate else 256,
    "win_length": 2048 if high_sample_rate else 1024,
    "n_mel_channels": 128 if high_sample_rate else 80,
    "mel_fmin": 0.0,
    "mel_fmax": None,
    "add_blank": True,
    "n_speakers": len(speakers) if len(speakers) > 1 else 2
  },
  "model": {
    "inter_channels": 192,
    "hidden_channels": 256,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5], [1,3,5], [1,3,5]],
    "upsample_rates": [8,8,4,2] if high_sample_rate else [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,4,4],
    "n_layers_q": 3,
    "use_spectral_norm": False
  },
  "speakers": speakers
}

if len(speakers) > 1:
  training_json["model"]["gin_channels"] = 256

import demjson
os.chdir('/content/sovits_f0_train/configs')
training_json_text = demjson.encode(training_json)
with open(json_filename, "w") as file:
  file.write(training_json_text)

os.chdir('/content/sovits_f0_train')



In [ ]:
#@title 预处理
os.chdir('/content/sovits_f0_train/monotonic_align')
!python setup.py build_ext --inplace
os.chdir('/content/sovits_f0_train')

若**使用预模型**（2.0只能用2.0的预模型，**下面打钩自动下载**），直接在预模型基础上训练，节省时间

使用预模型读取pth文件后，文件名的G_steps.pth，steps会乱是正常现象（使用v100训练，batch_size为32，colab显存不够改成了16，所以会乱掉，保存最新生成的模型即可）

程序读取一次预模型后，以后断点恢复训练、用**最新生成的一组pth继续训练**

In [ ]:
#@markdown 22050hz预模型,id为0-7可用
!mkdir /content/drive/MyDrive/nyarumul/
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/G_50000.pth -O /content/drive/MyDrive/nyarumul/G_50000.pth
  !wget https://huggingface.co/spaces/xiaolang/sovits_f0/resolve/main/D_50000.pth -O /content/drive/MyDrive/nyarumul/D_50000.pth

In [ ]:
#@title 训练

#@markdown 启用tensorboard可视化数据，首次运行可能十分钟左右出现数据

#@markdown epoch次数看/content/drive/MyDrive/模型名/train.log

#@markdown **首次**使用**预模型**时，成功运行约十五分钟后，发现自动保存了**新的.pth**（可能是G/D_20000.pth），然后**删除G/D_50000.pth**

#@markdown 模型名（使用预模型则为nyarumul）
hparams_model_name = "nyarumul"  # @param {type:"string"}
#@markdown 模型自动保存到 /云盘路径/模型名 ,记得定期清理空间！
param_enable_tb = True  # @param {type:"boolean"}
if param_enable_tb:
  #@markdown 云盘路径（一般不改） （定时清云盘！！！pth里保留最新的俩）
  logdir = "/content/drive/MyDrive/"  # @param {type:"string"}
  new_pth_dir = os.path.join(logdir, hparams_model_name)
  get_tensorboard_showing(new_pth_dir)
os.chdir('/content/sovits_f0_train')
#@markdown 配置文件json名（使用预模型则为nyarumul.json）
json_filename = "nyarumul.json"  # @param {type:"string"}
# 这里魔改过until.py的args
run_command_by_line(["python", "train_ms.py", "-c", "configs/{json}".format(json=json_filename), "-m", hparams_model_name, "-l",logdir])

In [ ]:
#@title 精简模型
import torch
#@markdown 模型名，数字是最新的模型steps
new_model_name = "G_207000.pth"  # @param {type:"string"}
checkpoint_dict = torch.load(f"{new_pth_dir}/{new_model_name}")
iteration = checkpoint_dict['iteration']
learning_rate = checkpoint_dict['learning_rate']
optimizer = checkpoint_dict['optimizer']
saved_state_dict = checkpoint_dict['model']
print(iteration)
#@markdown 输出xxx_epoch.pth；此模型**仅供合成使用**，去除训练信息、体积约为1/3，**无法训练使用**
torch.save({'model': saved_state_dict,
  'iteration': None,
  'optimizer': None,
  'learning_rate': None}, f'{new_pth_dir}/{iteration}_epochs.pth')